In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as mtick

In [ ]:
df=pd.read_csv("/kaggle/input/the-fincen-files/transactions_map.csv",index_col='id')

Set up datetime format and currency format

In [ ]:
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ncols=2
df['begin_date']=pd.to_datetime(df['begin_date'])
df['end_date']=pd.to_datetime(df['end_date'])
df['year']=df['begin_date'].dt.year

There are many missing values in the column of 'number of suspicious transactions' (ST), so I used the occurrence of ST instead. The function can work with datasets that have the same structure and variable names.

In [ ]:
def SumNumberYearlyST (df):
    labels_year, frequencies_year = np.unique(df['year'][~np.isnan(df['year'])], return_counts = True)
    #labels=labels_year.astype(int)
    plt.figure(figsize = (10,10))
    plt.plot(labels_year,frequencies_year)
    plt.title('Number of Suspicious Transactions per Year')
    plt.xticks(np.arange(min(labels_year), max(labels_year)+2, 2))
    plt.yticks(np.arange(min(frequencies_year)-1,max(frequencies_year),50))
    plt.xlabel('Year')
    plt.ylabel('Number of ST')
    plt.show()

In [ ]:
SumNumberYearlyST (df)

dire is the direction of stranscations, either 'originator' or 'beneficiary', top (int) is the highest ST occurrence.

In [ ]:
def NumberST(df, dire, top): 
    labels_org_con, frequencies_org_con =np.unique(df[dire+'_iso'], return_counts = True)
    count_sort_ind = np.argsort(-frequencies_org_con)
    plt.figure(figsize = (15,15))
    plt.bar(labels_org_con[count_sort_ind][:top],frequencies_org_con[count_sort_ind][:top])
    plt.title(f'Number of Suspicious Transactions in {dire} Country'.title())
    plt.yticks(np.arange(min(frequencies_org_con[count_sort_ind][:top])-20,max(frequencies_org_con),100))
    plt.xlabel(f'Top {top} {dire} Country'.title())
    plt.ylabel('Number of ST')
    plt.show()

In [ ]:
NumberST(df,'originator', 5)    

The highest ST amount

In [ ]:
def HighestST(df, dire, top):
    df1=(df.groupby([dire +'_iso'])['amount_transactions'].sum()/1000000).round(0).nlargest(top).reset_index()
    ax=df1.plot.bar(dire +'_iso', 'amount_transactions', fontsize=15,
                    figsize = (15,15),title=f'Amount of Suspicious Transactions in {dire} Country'.title(),
                    xlabel=f'Top {top} {dire} Country'.title(),ylabel='Amound of ST')
    ax.yaxis.set_major_formatter(tick) 

In [ ]:
HighestST(df,'beneficiary',5 )

The yearly highest ST amount

In [ ]:
##by transaction amount over year
def STyearly(df,dire, top):
    df1=(df.groupby([dire +'_iso'])['amount_transactions'].sum()/1000000).round(0).nlargest(top).reset_index()
    toplist=list(df1[dire+'_iso'])
    df2=df[['year', dire+'_iso','amount_transactions']].loc[df[dire+'_iso'].isin(toplist)]
    top_year=(df2.groupby([dire+'_iso','year'])['amount_transactions'].sum()/1000000).round(2).reset_index()
    top_year.set_index('year', inplace=True)
    top_year.index.astype(int)
    grouped = top_year.groupby(dire+'_iso')
    nrows = int(np.ceil(grouped.ngroups/ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15,15), sharex=True, sharey=True)
    for (key, ax) in zip(grouped.groups.keys(), axes.flatten()):
        grouped.get_group(key).plot(ax=ax)
        ax.legend([key],loc='upper right', frameon=False)
        ax.yaxis.set_major_formatter(tick) 
    ax.xaxis.set_major_locator(MaxNLocator(integer=True)) 
    plt.show() 

In [ ]:
STyearly(df,'beneficiary', 10)

Import geographical analysis packages

In [ ]:
##worldmaps of occurrence##
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import mapclassify as mc

Obtain world map

In [ ]:
world = gpd.read_file(gplt.datasets.get_path('world'))
world = world[world.id !='-99']

Setting up central points for countries

In [ ]:
world_points=world.copy()
world_points['centroid'] = world_points.centroid
world_points = world_points.set_geometry('centroid')

Generate multipoints for ST flow plots

In [ ]:
from shapely.geometry import MultiPoint
country_sum=(df.groupby(['originator_iso','beneficiary_iso'])['amount_transactions'].agg(sum)/100000).reset_index()
map_network = world_points.merge(country_sum, left_on="id", right_on="originator_iso")
map_network = world_points.merge(map_network, left_on="id", right_on="beneficiary_iso")

Basic map showing sum ST by country

In [ ]:
def mapST(df,dire):
    country_sum=df.groupby([dire+'_iso'])['amount_transactions'].agg(sum)/100000
    map_st = world.merge(country_sum, left_on="id", right_on=dire+'_iso')
    scheme = mc.UserDefined(map_st['amount_transactions'], bins=[2500, 5000, 10000, 50000])
    gplt.choropleth(
        map_st, hue='amount_transactions',
        edgecolor='lightgray', linewidth=1,
        cmap='rainbow', legend=True, legend_kwargs={'loc': 'lower left', 'fontsize':15},
        scheme=scheme, figsize=(15,15),
        legend_labels=['< $2,500 million','$2,500-5,000 million', '$5,000-10,000 million', '$10,000-50,000 million',
             '>$50,000 million'])
    plt.title("Sum of ST by Country between 2000 and 2017",fontsize=20)
    plt.show()

In [ ]:
mapST(df,'beneficiary')

Flow map showing ST sending out and coming in a country

In [ ]:
###ST Flows###    
def CountrySTFlow(df, dire, country, top):
    
    ###ST flowing in or out from a Country###
    df2=df[df[dire+'_iso']==country]
    df2['multi'] = [MultiPoint([x, y]) for x, y in zip(df2.centroid_y, df2.centroid_x)]
    df2=df2.set_geometry('multi')
    df2=df2.nlargest(top, columns=['amount_transactions'])
    scheme = mc.JenksCaspall(df2['amount_transactions'])
    lc=[f'${x:,.0f} million' for x in df2['amount_transactions']] 
    ax = gplt.sankey(df2, projection=gcrs.WebMercator(),
                hue='amount_transactions', scheme=scheme, cmap='Dark2', 
                legend=True,legend_kwargs={'loc': 'lower left', 'fontsize':10},
                legend_labels=lc, figsize=(12,12), linestyles=':')
    gplt.polyplot(world, ax=ax, facecolor='lightgray', edgecolor='white')
    plt.title(f"Sum of top {top} ST of {country} {dire.title()} between 2000 and 2017",fontsize=15)
    plt.show()

In [ ]:
CountrySTFlow(map_network, 'beneficiary', 'USA', 5)